# Causation In Observational Studies
source: https://www.youtube.com/watch?v=MrZDBsS7hG4&list=PLoazKTcS0Rzb6bb9L508cyJ1z-U9iWkA0&index=5

## Short Summary
To establish causality in observational studies (if possible), you have to control for confounding variables 

### Y - outcome, T - treatment, C - confounding variable

### $E[Y | do(T = t)] = E_C E[Y | t, C] = \sum_{c} E[Y | t, c]P(c)$

# A Simple Example - Simpson's Paradox

In [1]:
import numpy as np
import pandas as pd

In [15]:
# 210 / 1400 patients with mild conditions under treatment a died
treatment_a_mild = np.concatenate([np.ones(210) , np.zeros(1400 - 210)]) 

# 30 / 100 patients with severe conditions under treatment a died
treatment_a_severe  = np.concatenate([np.ones(30), np.zeros(100 - 30)])  

# 5 / 50 patients with mild conditions under treatment b died
treatment_b_mild  = np.concatenate([np.ones(5), np.zeros(50 - 5)])  

# 100 / 5000 patients with severe conditions under treatment b died
treatment_b_severe  = np.concatenate([np.ones(100), np.zeros(500 - 100)])  

data1 = pd.DataFrame(treatment_a_mild, columns=["outcome"])
data1["treatment"] = "A"
data1["condition"] = "mild"

data = pd.DataFrame()

for treatment, severity, outcome in (
    zip(["A","A","B","B"],
        ["mild", "severe", "mild", "severe"],
        [treatment_a_mild, 
         treatment_a_severe,
         treatment_b_mild,
         treatment_b_severe]
        )
):
    data_tmp = pd.DataFrame(outcome, columns=["outcome"])
    data_tmp["treatment"] = treatment
    data_tmp["severity"] = severity
    
    data = pd.concat([data_tmp, data])

print(data.shape) 
data.head()

(2050, 3)


,outcome,treatment,severity
0,1.0,B,severe
1,1.0,B,severe
2,1.0,B,severe
3,1.0,B,severe
4,1.0,B,severe


### Question: Which treatment had a lower death rate?

In [21]:
by_treatment = data.groupby("treatment")["outcome"].agg(["mean","count"]).reset_index()
by_treatment.columns = ["treatment", "death_rate", "num_observations"]
by_treatment

,treatment,death_rate,num_observations
0,A,0.160000,1500
1,B,0.190909,550


At a first glance, it seems like treatment A is better because the death rate is lower. However, we should immediately be skeptical because the sample size in treatment B is much smaller than A. More importantly, however, there is a confounding variable - disease severtiy.

In [26]:
by_treatment_sev = data.groupby(["treatment", "severity"])["outcome"].agg(["mean","count"]).reset_index()
by_treatment_sev.columns = ["treatment", "severity", "death_rate", "num_observations"]
by_treatment_sev

,treatment,severity,death_rate,num_observations
0,A,mild,0.15,1400
1,A,severe,0.30,100
2,B,mild,0.10,50
3,B,severe,0.20,500


The death rate for both mild and severe cases is lower in treatment B. The aggregate death rate in B is higher, however, because a much larger sample size fell into the severe category than in treatment A.


In [35]:
p_mild = (data["severity"] == "mild").mean()
p_severe = (data["severity"] == "severe").mean()

p_death_a_mild = data[(data["treatment"] == "A") & (data["severity"] == "mild")]["outcome"].mean()
p_death_a_severe = data[(data["treatment"] == "A") & (data["severity"] == "severe")]["outcome"].mean()

p_death_b_mild = data[(data["treatment"] == "B") & (data["severity"] == "mild")]["outcome"].mean()
p_death_b_severe = data[(data["treatment"] == "B") & (data["severity"] == "severe")]["outcome"].mean()

print("Treatment A causal death rate: ", p_death_a_mild*p_mild + p_death_a_severe*p_severe)
print("Treatment B causal death rate: ", p_death_b_mild*p_mild + p_death_b_severe*p_severe)

Treatment A causal death rate:  0.19390243902439025
Treatment B causal death rate:  0.12926829268292683
